In [1]:
from ucimlrepo import fetch_ucirepo 
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, Binarizer,KBinsDiscretizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

import pandas as pd

In [2]:
heart = fetch_ucirepo(id=45)

In [3]:
# data (as pandas dataframes) 
X = heart.data.features 
y = heart.data.targets
df = X
df["ill"] = np.array(y>0, dtype=int)

display(df.shape)
df = df.dropna()
display(df.shape)

print(cross_val_score(LogisticRegression(),df.loc[:, df.columns != 'ill'],df.ill).mean())

df.columns

(303, 14)

(297, 14)

0.8347457627118644


/home/sheid/.cache/pypoetry/virtualenvs/ijar-experiments-psl-vs-slim-s9rquOQ_-py3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/sheid/.cache/pypoetry/virtualenvs/ijar-experiments-psl-vs-slim-s9rquOQ_-py3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'ill'],
      dtype='object')

In [4]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,ill
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,57,0,4,140,241,0,0,123,1,0.2,2,0.0,7.0,1
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0,1
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0,1


In [5]:
column_trans = ColumnTransformer([
    ('label', "passthrough", ["ill"]),
    ('', KBinsDiscretizer(n_bins=3) , ["age","trestbps", "thalach", "chol"]),
    ('bin', KBinsDiscretizer(n_bins=2, encode="ordinal", strategy="kmeans") , ["cp", "restecg","oldpeak", "slope" ]),
],
verbose_feature_names_out=True)

column_trans.fit(df)

cols = column_trans.get_feature_names_out()
df_out = pd.DataFrame(column_trans.transform(df), columns=cols, dtype=int)
len(cols), cols, df_out.shape

/home/sheid/.cache/pypoetry/virtualenvs/ijar-experiments-psl-vs-slim-s9rquOQ_-py3.11/lib/python3.11/site-packages/sklearn/preprocessing/_discretization.py:248: FutureWarning: In version 1.5 onwards, subsample=200_000 will be used by default. Set subsample explicitly to silence this warning in the mean time. Set subsample=None to disable subsampling explicitly.
  warnings.warn(


(17,
 array(['label__ill', '__age_0.0', '__age_1.0', '__age_2.0',
        '__trestbps_0.0', '__trestbps_1.0', '__trestbps_2.0',
        '__thalach_0.0', '__thalach_1.0', '__thalach_2.0', '__chol_0.0',
        '__chol_1.0', '__chol_2.0', 'bin__cp', 'bin__restecg',
        'bin__oldpeak', 'bin__slope'], dtype=object),
 (297, 17))

In [6]:
df_out.to_csv("../heart.csv", index=False)
print(cross_val_score(LogisticRegression(),df_out.loc[:, df_out.columns != 'label__ill'],df_out.label__ill).mean())

df_out

0.7135593220338983


,label__ill,__age_0.0,__age_1.0,__age_2.0,__trestbps_0.0,__trestbps_1.0,__trestbps_2.0,__thalach_0.0,__thalach_1.0,__thalach_2.0,__chol_0.0,__chol_1.0,__chol_2.0,bin__cp,bin__restecg,bin__oldpeak,bin__slope
0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,1,1
1,1,0,0,1,0,0,1,1,0,0,0,0,1,1,1,0,1
2,1,0,0,1,1,0,0,1,0,0,0,1,0,1,1,1,1
3,0,1,0,0,0,1,0,0,0,1,0,1,0,1,0,1,1
4,0,1,0,0,0,1,0,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,0,1
293,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1
294,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,1,1
295,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1
